In [ ]:
import json
import open3d as o3d
from sklearn import cluster

from PointCloudDataFrame import PointCloudDataFrame

In [ ]:
pcd = o3d.io.read_point_cloud('point_cloud.pts')

In [ ]:
pcd_down = pcd.voxel_down_sample(voxel_size=0.001)  # 1mm
# o3d.visualization.draw_geometries([pcd_down])

In [ ]:
df = PointCloudDataFrame.from_pcd(pcd_down)

In [ ]:
df_filter1 = df[(df['s'] < 0.075) & (df['v'] > 0.2)]
df_filter1.write_pts('output/filter1.pts')
pcd_filter1 = df_filter1.to_pcd()
# o3d.visualization.draw_geometries([pcd_filter1])

In [ ]:
df_filter2 = df[(df['s'] < 0.075) & (df['v'] > 0.2) & (df['z'] > 0.1)]
df_filter2.write_pts('output/filter2.pts')
pcd_filter2 = df_filter2.to_pcd()
# o3d.visualization.draw_geometries([pcd_filter2])

In [ ]:
_, ind = pcd_filter2.remove_radius_outlier(nb_points=48, radius=0.005)
pcd_inliers = pcd_filter2.select_by_index(ind)
pcd_inliers.paint_uniform_color([0.8, 0.8, 0.8])
o3d.visualization.draw_geometries([pcd_inliers])

In [ ]:
model = cluster.KMeans(n_clusters=128, n_init='auto')
model.fit(pcd_inliers.points)

In [ ]:
dictionary = {str(i): list(k) for (i, k) in enumerate(model.cluster_centers_)}
json_object = json.dumps(dictionary)
with open('output/clusters.json', 'w') as f:
    f.write(json_object)

In [ ]:
clusters = o3d.geometry.PointCloud()
clusters.points = o3d.utility.Vector3dVector(model.cluster_centers_)
o3d.io.write_point_cloud('output/clusters.xyz', clusters, write_ascii=True)

In [ ]:
o3d.visualization.draw_geometries([pcd_inliers, clusters])